In [1]:
from transient_piezo_truss import * 
import matplotlib.pyplot as plt 

In [ ]:
# Create a static_piezo_truss object
M, N = 3, 2
freq = [5]
transient_piezo_truss = TransientPiezoTruss(size=(M, N), freq=freq)
# Plot the structure
UNodes_free = [0, 1, 2*M-1]
u_free = [[[0, 0]], [[0, 0]], [[0, 0]]]
#
UNodes_clamped = [0, 1, 4, 2*M-1,]
u_clamped = [[[0, 0]], [[0, 0]], [[1E-7, 0]], [[0, 0]]]
#
T = np.linspace(0, 1, 1000)
N = len(T) // 2
Vol = np.ones((len(transient_piezo_truss.edges), len(freq), 2)) * 10
Vol[:, :, 1] = 0
#
Epoch = 25
lr = 1E9
# Vol = np.ones((len(transient_piezo_truss.edges), len(freq), 2)) * 10
for i in tqdm.tqdm(range(Epoch)):
    #
    transient_piezo_truss.solve(T, UNodes_free, u_free, V=Vol)
    Q_free = transient_piezo_truss.calQ.copy()
    #
    transient_piezo_truss.solve(T, UNodes_clamped, u_clamped, V=Vol)
    Q_clamped = transient_piezo_truss.calQ.copy()
    #
    for n, fq in enumerate(freq):
        Q_free_sin = 2 * Q_free[:, N:] @ np.sin(fq * 2 * np.pi * T[N:]) / len(T[N:])
        Q_free_cos = 2 * Q_free[:, N:] @ np.cos(fq * 2 * np.pi * T[N:]) / len(T[N:])
        #
        Q_clamped_sin = 2 * Q_clamped[:, N:] @ np.sin(fq * 2 * np.pi * T[N:]) / len(T[N:])
        Q_clamped_cos = 2 * Q_clamped[:, N:] @ np.cos(fq * 2 * np.pi * T[N:]) / len(T[N:])
        #
        Vol[:, n, 0] += lr * (Q_clamped_sin - Q_free_sin)
        Vol[:, n, 1] += lr * (Q_clamped_cos - Q_free_cos)